# Renewal benchmark

This notebook allows to benchmark Renewal competitors. It can be enhanced and made real-time compatible.
We use TrueSkill as the eval system. We access buckt in MongoDB, hence you need to create an SSH tunnel:

    ssh -L 27017:localhost:27017 renewal@renewal01

To install dependencies, you can use this script: https://github.com/hayj/Bash/blob/master/hjupdate.sh

## Importing all packages

In [1]:
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *

In [2]:
from datetime import datetime as pdtime
import numpy as np
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

In [3]:
sys.path.append('../')
from renewalsimulator.utils import *

## Getting the records from the Mongo database

In [4]:
config = \
{
    # "start_time": "2021-09-16", # 2021-09-09
    "start_time": "2021-11-11", # Started the Wed, 10 Nov 2021 20:35:53
    "end_time": pdtime.now(), # pdtime.now()
}
config["start_time"] = convertDate(config["start_time"], DATE_FORMAT.datetime)
config["end_time"] = convertDate(config["end_time"], DATE_FORMAT.datetime)

In [5]:
print("Competition duration: " + secondsToHumanReadableDuration(convertDate(config["end_time"], DATE_FORMAT.timestamp) - convertDate(config["start_time"], DATE_FORMAT.timestamp)))
bp(config)

Competition duration: 370h 12m 44.894s
{ end_time: 2021-11-26 11:12:44.894139, start_time: 2021-11-11 00:00:00+00:00 }


In [6]:
bucketsCol = MongoCollection("renewal", "recommendation_buckets")

renewal recommendation_buckets initialised.


In [7]:
buckets = list(bucketsCol.find({"timestamp": {"$gt": config["start_time"], "$lt": config["end_time"]}}))
buckets = [b for b in buckets if "interleaving" in b]
bp(buckets)
print("Found " + str(len(buckets)) + " buckets")

[
  {
    _id: 618cbef51763b365bf7cccde,
    bucket_id: 1848,
    interleaving: 
    {
      interleaved_recommendations: [ 977577, 983005, ..., 974702, 981197 ],
      interleaving_method: team-draft,
      recsystems: 
      [
        {
          recommendations: [ 979776, 983073, ..., 981532, 980383 ],
          recsystem_id: 6149ad143f9819ee0c435b8f
        },
        {
          recommendations: [ 978816, 975812, ..., 983005, 982335 ],
          recsystem_id: 60bddeef82145624b8a06523
        }
      ]
    },
    recommendations: 
    [
      {
        recommendations: [ 810274, 975056, ..., 1001350, 1001349 ],
        recsystem_id: 5fae9ec12573bc71bec43bcc,
        response_time: 167.55,
      },
      {
        recommendations: [ 983005, 976487, ..., 982165, 980573 ],
        recsystem_id: 60bddeef82145624b8a06523,
        response_time: 1257.06,
      },
      {
        recommendations: [ 977577, 980383, ..., 976931, 976930 ],
        recsystem_id: 6149ad143f9819ee0c435b8f,
    

In [8]:
def getClicksCursor(test=False):
    clicksCol = MongoCollection("renewal", "articles.events", verbose=False)
    return clicksCol.find\
    (
        {
            'clicked': None if test else True,
            "timestamp":
            {
                "$gt": config["start_time"],
                "$lt": config["end_time"]
            }
        },
        sort=[('timestamp', pymongo.ASCENDING)],
    )

In [9]:
recsysNameCache = dict()
def getRecsysName(objid):
    if objid in recsysNameCache:
        result = recsysNameCache[objid]
    else:
        col = MongoCollection("renewal", "recsystems", verbose=False)
        result = col.findOne(ObjectId(objid))["name"]
        recsysNameCache[objid] = result
    if result == "g0":
        return "baseline-keywords"
    elif result.startswith("baseline-random"):
        return "baseline-random"
    elif result.startswith("baseline-popular"):
        return "baseline-popular"
    elif result.startswith("g3"):
        return "team-1"
    elif result.startswith("g6"):
        return "team-2"
    elif result.startswith("g11"):
        return "team-3"
    else:
        return result

Currently, we consider a user as active if he clicked on, at least, 3 news. In the future, we'll consider a user as active knowing the number of buckets or the number of days he got a new bucket...

In [10]:
min_clicked_news = 3
clickedNews = dict()
for current in getClicksCursor(test=False):
    if current['user_id'] not in clickedNews:
        clickedNews[current['user_id']] = set()
    clickedNews[current['user_id']].add(current['article_id'])
users_count = len(clickedNews)
print("Here all users (including not active):")
for user, clicks in clickedNews.items():
    print(user + ": " + str(len(clicks)))
for user in list(clickedNews.keys()):
    if len(clickedNews[user]) < min_clicked_news:
        del clickedNews[user]
active_users_count = len(clickedNews)

Here all users (including not active):
eu5jv2XxGqUpACPibwRoxnjjTHq1: 75
tQ4DKUGSfvOZtuU81uvfINrh4y83: 108
SnIlEEVDzzewFaXkUydr2K6vEpU2: 5
5J1RfJiTq0OJA6b5UFUeTBgX06j1: 11
zY1q2k013nOFlEjHDz5eliGGMNZ2: 4
UC5W7D1OuiN0YkHNsGoewjBViRI2: 44
XkqRfUiFmwUb48NqMuqi5urj3LF3: 48
S3oxvJ9rUiMLNGl4YmZ2bGBsmD92: 1
Mhkc4xuaFPWnmbFomIv8drAtsn13: 1
C88Z7JqrqoRnO7uvJtBBvBD3oTY2: 3
4rlAJaP6AchXCkPEX8TZeqOLUuN2: 10
7P3QOs9x5lgnQuQgGzIGTJK1f3A3: 2
A1f1TXyxTFWUJxrietIUIg6wgtd2: 1
N1UkpI2c6ZddzCI9oXprweB3TNo2: 1
A604UnXcaLhwXXM4DeufZUXlfG83: 1
H2iybE0qvDeEuwVZmKHtINw7uBv1: 1
9Y0byFS0bogPhE4Oit02qEL5oYm2: 1
QU9LGpl0ziOI4LfZYC0YSR3gRGh1: 4
OE2v74G9RhXqn7Wp5nl649qAEhr2: 7
YOgF8ICD2yXVLmqtJIdQrCHMYiD2: 1
d2M0BHxGcBSsGRuLXpCvJ8E3FTI2: 2
HOR220HHMLfKbsVwallfDGQk31K2: 1
CAGeiYJ4DiSJ38OEtRcKTWi27qj2: 3
fSFK5HdAhCNri8qBY3pmh09TgUL2: 1
JnlyPC82aeVM8ZkHkkMlJEJHDLg1: 5
x7XwhYsieTb7AaxvrU6KCdUj8AF3: 1
lphfFTeWfMWVfnp8xuUPQMavxZw1: 2


In [11]:
print("Here every active users:")
for user, clicks in clickedNews.items():
    print(user + ": " + str(len(clicks)))

Here every active users:
eu5jv2XxGqUpACPibwRoxnjjTHq1: 75
tQ4DKUGSfvOZtuU81uvfINrh4y83: 108
SnIlEEVDzzewFaXkUydr2K6vEpU2: 5
5J1RfJiTq0OJA6b5UFUeTBgX06j1: 11
zY1q2k013nOFlEjHDz5eliGGMNZ2: 4
UC5W7D1OuiN0YkHNsGoewjBViRI2: 44
XkqRfUiFmwUb48NqMuqi5urj3LF3: 48
C88Z7JqrqoRnO7uvJtBBvBD3oTY2: 3
4rlAJaP6AchXCkPEX8TZeqOLUuN2: 10
QU9LGpl0ziOI4LfZYC0YSR3gRGh1: 4
OE2v74G9RhXqn7Wp5nl649qAEhr2: 7
CAGeiYJ4DiSJ38OEtRcKTWi27qj2: 3
JnlyPC82aeVM8ZkHkkMlJEJHDLg1: 5


## Creating all outcomes that will feed a RenewalRanking instance

In [12]:
outcomes = []
competitors = set()
no_match_count = 0
match_count = 0
for bucket in buckets:
    try:
        del bucket['articles']
    except: pass
    user_id = bucket["user_id"]
    if user_id not in clickedNews:
        clickedNews[user_id] = set()
    timestamp = bucket["timestamp"]
    interleaved_list = bucket["interleaving"]["interleaved_recommendations"]
    recommendations = dict()
    for current in bucket['interleaving']['recsystems']:
        recommendations[current['recsystem_id']] = current['recommendations']
    assert len(recommendations) == 2
    recsys_a, recsys_b = tuple(recommendations.keys())
    competitors.add(recsys_a)
    competitors.add(recsys_b)
    recs_a, recs_b = recommendations[recsys_a], recommendations[recsys_b]
    assert recsys_a != recsys_b
    a_clicks = 0
    b_clicks = 0
    for article_id in interleaved_list:
        if article_id in clickedNews[user_id]:
            if article_id in recs_a:
                a_clicks += 1
            if article_id in recs_b:
                b_clicks += 1
    if a_clicks == 0 and b_clicks == 0:
        pass
        no_match_count += 1
        # print("No match")
    elif a_clicks > b_clicks:
        match_count += 1
        print(getRecsysName(recsys_a) + " won against " + getRecsysName(recsys_b) + " (" + str(timestamp) + ")")
        outcomes.append((recsys_a, recsys_b, -1, timestamp))
    elif a_clicks < b_clicks:
        match_count += 1
        print(getRecsysName(recsys_b) + " won against " + getRecsysName(recsys_a) + " (" + str(timestamp) + ")")
        outcomes.append((recsys_a, recsys_b, 1, timestamp))
    else:
        match_count += 1
        print("It's a draw for " + getRecsysName(recsys_a) + " and " + getRecsysName(recsys_b) + " (" + str(timestamp) + ")")
        outcomes.append((recsys_a, recsys_b, 0, timestamp))
competitors = list(competitors)
bp(competitors, 4)
bp(outcomes, 4)
print("no_match_count: " + str(no_match_count))
print("match_count: " + str(match_count))

team-3 won against baseline-keywords (2021-11-11 06:57:54.739000)
baseline-popular won against baseline-random (2021-11-11 07:00:31.750000)
baseline-random won against team-2 (2021-11-11 07:02:11.070000)
It's a draw for baseline-keywords and baseline-random (2021-11-11 09:33:01.250000)
baseline-popular won against team-2 (2021-11-11 13:32:22.903000)
baseline-random won against baseline-popular (2021-11-11 14:18:28.199000)
It's a draw for baseline-keywords and baseline-popular (2021-11-11 14:54:43.387000)
baseline-random won against baseline-keywords (2021-11-11 14:54:47.147000)
team-2 won against baseline-keywords (2021-11-11 15:56:24.889000)
team-3 won against baseline-keywords (2021-11-11 16:02:32.463000)
baseline-keywords won against baseline-popular (2021-11-11 16:12:34.300000)
team-3 won against team-2 (2021-11-11 18:20:36.654000)
team-3 won against baseline-random (2021-11-12 11:07:48.586000)
team-2 won against baseline-keywords (2021-11-12 13:16:28.803000)
It's a draw for baseli

## Evaluation using the `RenewalRanking` class

In [13]:
rr = RenewalRanking(competitors, float_precision=1)
rankings = []
for a, b, result, timestamp in outcomes:
    if result == 1:
        winner = b
    elif result == -1:
        winner = a
    else:
        winner = None
    rr.match(a, b, winner=winner)
    rankings.append((rr.get_ranking_stats(), timestamp))
bp(rankings)

[
  (
    [ ( 60bddeef82145624b8a06523, 29.3, 7.17 ), ( 5fae9ec12573bc71bec43bcc, 25.0, 8.33 ), ..., ( 5fae9ebf2573bc71bec43bca, 25.0, 8.33 ), ( 6149ad143f9819ee0c435b8f, 20.6, 7.17 ) ],
    2021-11-11 06:57:54.739000
  ),
  (
    [ ( 60bddeef82145624b8a06523, 29.3, 7.17 ), ( 5fae9ec12573bc71bec43bcc, 29.3, 7.17 ), ..., ( 6149ad143f9819ee0c435b8f, 20.6, 7.17 ), ( 5fae9ebf2573bc71bec43bca, 20.6, 7.17 ) ],
    2021-11-11 07:00:31.750000
  ),
  ...,
  (
    [ ( 60bde65882145624b8a06525, 26.4, 2.15 ), ( 60bddeef82145624b8a06523, 24.9, 1.16 ), ..., ( 60bde68382145624b8a06526, 23.7, 0.98 ), ( 5fae9ec12573bc71bec43bcc, 23.2, 1.08 ) ],
    2021-11-26 08:30:55.721000
  ),
  (
    [ ( 60bde65882145624b8a06525, 26.4, 2.15 ), ( 60bddeef82145624b8a06523, 24.9, 1.16 ), ..., ( 60bde68382145624b8a06526, 23.9, 0.98 ), ( 5fae9ec12573bc71bec43bcc, 23.1, 1.07 ) ],
    2021-11-26 08:32:43.249000
  )
]


In [14]:
# Converting it in dict:
rk = []
for i in range(len(outcomes)):
    current = dict()
    current["recsys_a"] = outcomes[i][0]
    current["recsys_b"] = outcomes[i][1]
    current["result"] = outcomes[i][2]
    current["timestamp"] = outcomes[i][3]
    current["ranking"] = rankings[i][0]
    rk.append(current)
bp(rk)

[
  {
    ranking: [ ( 60bddeef82145624b8a06523, 29.3, 7.17 ), ( 5fae9ec12573bc71bec43bcc, 25.0, 8.33 ), ..., ( 5fae9ebf2573bc71bec43bca, 25.0, 8.33 ), ( 6149ad143f9819ee0c435b8f, 20.6, 7.17 ) ],
    recsys_a: 6149ad143f9819ee0c435b8f,
    recsys_b: 60bddeef82145624b8a06523,
    result: 1,
    timestamp: 2021-11-11 06:57:54.739000
  },
  {
    ranking: [ ( 60bddeef82145624b8a06523, 29.3, 7.17 ), ( 5fae9ec12573bc71bec43bcc, 29.3, 7.17 ), ..., ( 6149ad143f9819ee0c435b8f, 20.6, 7.17 ), ( 5fae9ebf2573bc71bec43bca, 20.6, 7.17 ) ],
    recsys_a: 5fae9ec12573bc71bec43bcc,
    recsys_b: 5fae9ebf2573bc71bec43bca,
    result: -1,
    timestamp: 2021-11-11 07:00:31.750000
  },
  ...,
  {
    ranking: [ ( 60bde65882145624b8a06525, 26.4, 2.15 ), ( 60bddeef82145624b8a06523, 24.9, 1.16 ), ..., ( 60bde68382145624b8a06526, 23.7, 0.98 ), ( 5fae9ec12573bc71bec43bcc, 23.2, 1.08 ) ],
    recsys_a: 60bde68382145624b8a06526,
    recsys_b: 60bddeef82145624b8a06523,
    result: -1,
    timestamp: 2021-11-26 08

In [15]:
rk = []
for ranking, timestamp in rankings:
    current_rk = dict()
    for current_competitor in ranking:
        current_rk[current_competitor[0]] = dict()
        current_rk[current_competitor[0]]["score"] = current_competitor[1]
        current_rk[current_competitor[0]]["confidence"] = current_competitor[2]
    rk.append(current_rk)
bp(rk)

[
  {
    5fae9ebf2573bc71bec43bca: { confidence: 8.33, score: 25.0 },
    5fae9ec12573bc71bec43bcc: { confidence: 8.33, score: 25.0 },
    60bddeef82145624b8a06523: { confidence: 7.17, score: 29.3 },
    60bde65882145624b8a06525: { confidence: 8.33, score: 25.0 },
    60bde68382145624b8a06526: { confidence: 8.33, score: 25.0 },
    6149ad143f9819ee0c435b8f: { confidence: 7.17, score: 20.6 }
  },
  {
    5fae9ebf2573bc71bec43bca: { confidence: 7.17, score: 20.6 },
    5fae9ec12573bc71bec43bcc: { confidence: 7.17, score: 29.3 },
    60bddeef82145624b8a06523: { confidence: 7.17, score: 29.3 },
    60bde65882145624b8a06525: { confidence: 8.33, score: 25.0 },
    60bde68382145624b8a06526: { confidence: 8.33, score: 25.0 },
    6149ad143f9819ee0c435b8f: { confidence: 7.17, score: 20.6 }
  },
  ...,
  {
    5fae9ebf2573bc71bec43bca: { confidence: 1.09, score: 24.6 },
    5fae9ec12573bc71bec43bcc: { confidence: 1.08, score: 23.2 },
    60bddeef82145624b8a06523: { confidence: 1.16, score: 24.9

## Creating intermediare plots

In [16]:
x = []
x_translation = dict()
for ranking_stats, dtime in rankings:
    timestamp = convertDate(dtime, DATE_FORMAT.timestamp)
    x.append(dtime)
    x_translation[timestamp] = str(dtime)
bp(x)
bp(x_translation)

[ 2021-11-11 06:57:54.739000, 2021-11-11 07:00:31.750000, ..., 2021-11-26 08:30:55.721000, 2021-11-26 08:32:43.249000 ]
{ 1636610274.739: 2021-11-11 06:57:54.739000, 1636610431.75: 2021-11-11 07:00:31.750000, 1636610531.07: 2021-11-11 07:02:11.070000, 1636619581.25: 2021-11-11 09:33:01.250000, 1636633942.903: 2021-11-11 13:32:22.903000, ..., 1637875720.938: 2021-11-25 22:28:40.938000, 1637910243.599: 2021-11-26 08:04:03.599000, 1637911462.599: 2021-11-26 08:24:22.599000, 1637911855.721: 2021-11-26 08:30:55.721000, 1637911963.249: 2021-11-26 08:32:43.249000 }


In [17]:
scores = dict()
for competitor in competitors:
    current_scores = []
    for ranking in rankings:
        for current in ranking[0]:
            if current[0] == competitor:
                current_scores.append(current[1])
                break
    scores[str(competitor)] = current_scores
bp(scores)

{
  5fae9ebf2573bc71bec43bca: [ 25.0, 20.6, ..., 24.6, 24.6 ],
  5fae9ec12573bc71bec43bcc: [ 25.0, 29.3, ..., 23.2, 23.1 ],
  60bddeef82145624b8a06523: [ 29.3, 29.3, ..., 24.9, 24.9 ],
  60bde65882145624b8a06525: [ 25.0, 25.0, ..., 26.4, 26.4 ],
  60bde68382145624b8a06526: [ 25.0, 25.0, ..., 23.7, 23.9 ],
  6149ad143f9819ee0c435b8f: [ 20.6, 20.6, ..., 24.2, 24.2 ]
}


In [18]:
data = mergeDicts(scores, dict(x=x))
original_data = copy.deepcopy(data)
bp(data)

{
  5fae9ebf2573bc71bec43bca: [ 25.0, 20.6, ..., 24.6, 24.6 ],
  5fae9ec12573bc71bec43bcc: [ 25.0, 29.3, ..., 23.2, 23.1 ],
  60bddeef82145624b8a06523: [ 29.3, 29.3, ..., 24.9, 24.9 ],
  60bde65882145624b8a06525: [ 25.0, 25.0, ..., 26.4, 26.4 ],
  60bde68382145624b8a06526: [ 25.0, 25.0, ..., 23.7, 23.9 ],
  6149ad143f9819ee0c435b8f: [ 20.6, 20.6, ..., 24.2, 24.2 ],
  x: [ 2021-11-11 06:57:54.739000, 2021-11-11 07:00:31.750000, ..., 2021-11-26 08:30:55.721000, 2021-11-26 08:32:43.249000 ]
}


In [19]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, output_notebook, show

In [20]:
output_notebook()

Loading BokehJS ...

In [21]:
source = ColumnDataSource(data=data)

In [22]:
p = figure(plot_width=1000, plot_height=600, x_axis_type='datetime')

In [23]:
# p.xaxis.ticker = x
# p.xaxis.major_label_overrides = x_translation

In [24]:
from bokeh.palettes import Category10
colors = Category10[10]

In [25]:
legend_it = []
i = 0
for current in scores.keys():
    print(current)
    el = p.line('x', current, source=source, line_width=2, color=colors[i])
    legend_it.append((getRecsysName(current), [el]))
    i += 1
    if i == len(colors):
        i = 0

60bddeef82145624b8a06523
5fae9ec12573bc71bec43bcc
60bde65882145624b8a06525
60bde68382145624b8a06526
6149ad143f9819ee0c435b8f
5fae9ebf2573bc71bec43bca


In [26]:
# from bokeh.models import SingleIntervalTicker, LinearAxis

In [27]:
# ticker = SingleIntervalTicker(interval=200, num_minor_ticks=10)
# xaxis = LinearAxis(ticker=ticker)
# xaxis.major_label_overrides = x_translation
# p.add_layout(xaxis, 'below')
# p.xaxis.major_label_orientation = math.pi/2

In [28]:
# x_translation

In [29]:
from bokeh.models import DatetimeTickFormatter
from bokeh.models import Legend

In [30]:
# p.xaxis.formatter=DatetimeTickFormatter(
#         hours=["%d %B %Y"],
#         days=["%d %B %Y"],
#         months=["%d %B %Y"],
#         years=["%d %B %Y"],
#     )

In [31]:
p.xaxis.major_label_orientation = math.pi/4

In [32]:
legend = Legend(items=legend_it)
legend.click_policy = "mute"
p.add_layout(legend, 'below')

In [33]:
show(p)

In [34]:
new_data = copy.deepcopy(data)
new_data['x'] = list(range(len(data['x'])))
p = figure(plot_width=1000, plot_height=600)
source = ColumnDataSource(data=new_data)
legend_it = []
i = 0
for current in scores.keys():
    print(current)
    el = p.line('x', current, source=source, line_width=2, color=colors[i])
    legend_it.append((getRecsysName(current), [el]))
    i += 1
    if i == len(colors):
        i = 0
legend = Legend(items=legend_it)
legend.click_policy = "mute"
p.add_layout(legend, 'below')
show(p)

60bddeef82145624b8a06523
5fae9ec12573bc71bec43bcc
60bde65882145624b8a06525
60bde68382145624b8a06526
6149ad143f9819ee0c435b8f
5fae9ebf2573bc71bec43bca


In [35]:
# TODO faire des points qui indique le 1vs1 en overlay
# TODO ajouter la date à interval regulier, ou tracer une ligne vertical avec date en abcsisse
# En overlay donner le Elo score et le score de confidence
# Print un tableau du ranking aussi

In [36]:
bp(new_data)

{
  5fae9ebf2573bc71bec43bca: [ 25.0, 20.6, ..., 24.6, 24.6 ],
  5fae9ec12573bc71bec43bcc: [ 25.0, 29.3, ..., 23.2, 23.1 ],
  60bddeef82145624b8a06523: [ 29.3, 29.3, ..., 24.9, 24.9 ],
  60bde65882145624b8a06525: [ 25.0, 25.0, ..., 26.4, 26.4 ],
  60bde68382145624b8a06526: [ 25.0, 25.0, ..., 23.7, 23.9 ],
  6149ad143f9819ee0c435b8f: [ 20.6, 20.6, ..., 24.2, 24.2 ],
  x: [ 0, 1, ..., 139, 140 ]
}


In [37]:
windows_length = 21 # 9, 51
polyorder = 3 # 2, 3
new_data2 = copy.deepcopy(new_data)
for key in new_data2:
    if key != 'x':
        # scipy.signal.savgol_filter(x, window_length, polyorder, ...)
        y = savgol_filter(new_data2[key], windows_length, polyorder) # 51, 3
        new_data2[key] = y

In [38]:
assigned_colors = dict()
color_index = 0
for recsys_id in sorted(new_data2):
    if len(recsys_id) > 2:
        assigned_colors[recsys_id] = colors[color_index]
        color_index += 1
bp(assigned_colors)

{ 5fae9ebf2573bc71bec43bca: #1f77b4, 5fae9ec12573bc71bec43bcc: #ff7f0e, 60bddeef82145624b8a06523: #2ca02c, 60bde65882145624b8a06525: #d62728, 60bde68382145624b8a06526: #9467bd, 6149ad143f9819ee0c435b8f: #8c564b }


In [39]:
p = figure(plot_width=1000, plot_height=600)
source = ColumnDataSource(data=new_data2)
legend_it = []
i = 0
for current in scores.keys():
    el = p.line('x', current, source=source, line_width=2, color=assigned_colors[current])
    legend_it.append((getRecsysName(current), [el]))
    i += 1
    if i == len(colors):
        i = 0
legend = Legend(items=legend_it)
legend.click_policy = "mute"
p.xaxis.axis_label = 'Interleaved versus (chronologically ordered)'
p.yaxis.axis_label = 'Elo score'
p.add_layout(legend, 'above')
show(p)

## Creating the final plot

In [40]:
# We calculate corresponding days for buckets:
all_days_str = []
all_days_datetime = copy.deepcopy(original_data['x'])
all_days_to_keep = [0] * len(all_days_datetime)
current_day = None
index = 0
for current in original_data['x']:
    current = current.strftime('%Y-%m-%d')
    all_days_str.append(current)
    if current_day is None or current_day != current:
        all_days_to_keep[index] = 1
        current_day = current
    index += 1
bp(all_days_str)
bp(all_days_datetime)
print(all_days_to_keep)

[ 2021-11-11, 2021-11-11, ..., 2021-11-26, 2021-11-26 ]
[ 2021-11-11 06:57:54.739000, 2021-11-11 07:00:31.750000, ..., 2021-11-26 08:30:55.721000, 2021-11-26 08:32:43.249000 ]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]


In [41]:
bp(new_data2)

{
  5fae9ebf2573bc71bec43bca: [22.20490307 23.28121589 24.24500084 25.09795107 25.8417597  26.47811987
   27.00872471 27.43526735 27.75944093 27.98293857 28.10745342 28.2822491
   27.99525989 27.84798954 27.46815953 26.92945407 26.55210853 26.14207257
   25.83285387 25.55796012 25.32915986 25.11909121 24.93331154 24.78640078
   24.71386074 24.76113109 24.92853874 25.21641059 25.32732919 25.53455378
   25.83121935 26.18188951 26.49012749 26.74007846 26.90536123 26.98793723
   27.03448839 26.94946061 26.73059823 26.50382478 26.19957502 25.84841451
   25.47263812 25.11997385 24.81082053 24.54694345 24.2904544  24.08855835
   23.93000981 23.86070611 23.8904544  23.91863354 23.93883622 23.97564564
   23.99709055 24.04262831 24.09519451 24.08358941 24.09601177 24.18571429
   24.23314809 24.28336058 24.31392612 24.31670481 24.35916966 24.32608696
   24.25197777 24.15714286 24.04759725 23.9607388  23.88241255 23.80653808
   23.76528277 23.70660347 23.65053939 23.60035959 23.541288   23.4815299

In [42]:
all_scores = set()
for key in new_data2:
    if key != 'x':
        for score in new_data2[key]:
            all_scores.add(score)
y_min = min(all_scores)
print(y_min)
y_max = max(all_scores)
print(y_max)

18.861082548267927
36.3318077803198


In [43]:
from bokeh.models import Label

In [44]:
p = figure(plot_width=1000, plot_height=600)
source = ColumnDataSource(data=new_data2)
legend_it = []
i = 0
for current in scores.keys():
    el = p.line('x', current, source=source, line_width=2, color=assigned_colors[current])
    legend_it.append((getRecsysName(current), [el]))
    i += 1
    if i == len(colors):
        i = 0
day_skip = 3
current_skip = 0
for i in range(len(all_days_to_keep)):
    to_keep = all_days_to_keep[i]
    day_str = all_days_str[i]
    if to_keep == 1:
        if current_skip == 0:
            p.line([i, i], [y_min, y_max], line_width=2, color="#cccccc")
            p.add_layout(Label(x=i, y=y_max, text=day_str, angle=math.pi/2, y_offset=-80, x_offset=-2, text_color="#aaaaaa"))
        current_skip += 1
        if current_skip == day_skip:
            current_skip = 0
legend = Legend(items=legend_it)
legend.click_policy = "mute"
p.xaxis.axis_label = 'Interleaved rec versus (chronologically ordered)'
p.yaxis.axis_label = 'Elo score'
p.add_layout(legend, 'above')
show(p)

In [45]:
print("Nombre d’utilisateurs actifs à ce jour : " + str(active_users_count))
print("Nombre d’utilisateurs non actifs à ce jour : " + str(users_count - active_users_count) + " (qui n'ont pas suffisamment d'interactions pour être pris en compte dans la compétition)")

Nombre d’utilisateurs actifs à ce jour : 13
Nombre d’utilisateurs non actifs à ce jour : 14 (qui n'ont pas suffisamment d'interactions pour être pris en compte dans la compétition)


In [46]:
# TODO in order to be sure the top system is really the best, we can change the outcomes of its first 1vs1, so it will start with a very low score and we'll see if he anyway get the same rank at the end...

## Creating the users activity plot

abscisse nombre de jours et ordonné nombre de buckets par jour en moyenne sur une fenêtre glissante de 3

In [47]:
# TODO use the previous data structure